In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.core.compute import AksCompute
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import ContainerImage
import numpy as np
import argparse
import os
import json


from azureml.core.authentication import ServicePrincipalAuthentication


SDK version: 1.9.0


In [2]:
tenant_id = 'e65922f9-4bd4-4f11-b1a3-48e89d75674e'
application_id = '423b1df1-5286-4955-a6e7-fc416aea35d4'
app_secret = 'v3d43JhbJ1x.A5mJ3hyS2GRD.O_ZC~FE6~'
workspace_name = 'MLOps_template_ML'
subscription_id = 'b198933e-f055-498f-958d-0726ab11eddb'
resource_group = 'MLOps_template'
aks_service_name = 'mnist-webapp-service-prod'

In [3]:
service_principal = ServicePrincipalAuthentication(
        tenant_id=tenant_id,
        service_principal_id=application_id,
        service_principal_password=app_secret)

ws = Workspace.get(
            name=workspace_name,
            subscription_id=subscription_id,
            resource_group=resource_group,
            auth=service_principal)

aks_service = Webservice(name = aks_service_name, workspace = ws)


In [15]:
test_data = np.load('./TestDeployment/test_data.npy')

test_samples = json.dumps({"data": test_data.tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = aks_service.run(input_data=test_samples)

print(result)


[0, 1, 2, 4, 8, 1, 3, 6, 2, 2, 0, 8, 9, 8, 3, 6, 4, 2, 5, 2, 7, 1, 1, 4, 7, 6, 3, 3, 9, 4]


In [16]:
print(aks_service.scoring_uri)
print(aks_service.get_keys())

http://52.229.21.121:80/api/v1/service/mnist-webapp-service-prod/score
('qSxvx33RBksICBtEaOGDy54s5iF1VTnz', 'OaJA90ArydxVvtG4kDwdLcc8LOkN0aV9')


### Activate telemetry
We are going to activate telemetry on the deployed model. Then we will make another call which will end up being logged by both Application Insights and data collection.

In [58]:
web_service = Webservice(ws, aks_service_name)

In [59]:
web_service.update(enable_app_insights=True, collect_model_data=True)

In [60]:
print(aks_service.run(input_data=test_samples))
print(aks_service.run(input_data=test_samples))
print(aks_service.run(input_data=test_samples))
print(aks_service.run(input_data=test_samples))

[0, 1, 2, 4, 8, 1, 3, 6, 2, 2, 0, 8, 9, 8, 3, 6, 4, 2, 5, 2, 7, 1, 1, 4, 7, 6, 3, 3, 9, 4]
[0, 1, 2, 4, 8, 1, 3, 6, 2, 2, 0, 8, 9, 8, 3, 6, 4, 2, 5, 2, 7, 1, 1, 4, 7, 6, 3, 3, 9, 4]
[0, 1, 2, 4, 8, 1, 3, 6, 2, 2, 0, 8, 9, 8, 3, 6, 4, 2, 5, 2, 7, 1, 1, 4, 7, 6, 3, 3, 9, 4]
[0, 1, 2, 4, 8, 1, 3, 6, 2, 2, 0, 8, 9, 8, 3, 6, 4, 2, 5, 2, 7, 1, 1, 4, 7, 6, 3, 3, 9, 4]
